In [1]:
from tinybig.config import config
from tinybig.util import set_random_seed

print('loading configs...')
config_file_name = 'iris_configs'
config_obj = config()
config = config_obj.load_yaml(cache_dir='./configs', config_file=config_file_name + '.yaml')
print(config)

loading configs...
{'configurations': {'device': 'mps', 'random_seed': 1234, 'data_configs': {'data_class': 'tinybig.data.iris', 'data_parameters': {'name': 'iris', 'train_batch_size': 1000, 'test_batch_size': 1000}}, 'learner_configs': {'learner_class': 'tinybig.learner.backward_learner', 'learner_parameters': {'name': 'error_backward_propagation', 'n_epochs': 1000, 'optimizer_configs': {'optimizer_class': 'torch.optim.AdamW', 'optimizer_parameters': {'lr': 0.03, 'weight_decay': 1e-05}}, 'loss_configs': {'loss_class': 'torch.nn.MSELoss', 'loss_parameters': {'reduction': 'mean'}}}}, 'model_configs': {'model_class': 'tinybig.model.rpn', 'model_parameters': {'device': 'mps', 'name': 'reconciled_polynomial_network', 'depth': 1, 'depth_alloc': [1], 'layer_configs': [{'layer_class': 'tinybig.module.rpn_layer', 'layer_parameters': {'name': 'rpn_layer', 'm': 4, 'n': 1, 'width': 1, 'width_alloc': [1], 'head_configs': [{'head_class': 'tinybig.module.rpn_head', 'head_parameters': {'l': None, 'ch

In [2]:
print('setting up environments...')
device = config['configurations'].pop('device')
random_seed = config['configurations'].pop('random_seed')
set_random_seed(random_seed)
print('device: ', device, '; random_seed: ', random_seed)

setting up environments...
device:  mps ; random_seed:  1234


In [3]:
print('instantiating objects from config...')
data_obj, model_obj, learner_obj, metric_obj, result_obj = [config_obj.instantiation_from_configs(config['configurations'][f'{stem}_configs'], device=device, class_name=f'{stem}_class', parameter_name=f'{stem}_parameters') for stem in ['data', 'model', 'learner', 'metric', 'output']]
print('parameter num: ', sum([parameter.numel() for parameter in model_obj.parameters()]))

instantiating objects from config...
parameter num:  8


In [4]:
print('loading dataset...')
data_loader = data_obj.load(train_percentage=0.9, normalize_X=True)

loading dataset...


In [5]:
print('training model...')
training_records = learner_obj.train(model=model_obj, data_loader=data_loader, device=device, metric=metric_obj, disable_tqdm=True, display_step=100)
model_obj.save_ckpt(cache_dir='./ckpt', checkpoint_file=f'{config_file_name}_checkpoint')

training model...
Epoch: 0, Test Loss: 2.2529475688934326, Test Score: 2.2529475778338415, Time Cost: 0.07504582405090332
Epoch: 100, Test Loss: 0.07801901549100876, Test Score: 0.0780190096300517, Time Cost: 0.0018749237060546875
Epoch: 200, Test Loss: 0.06200331822037697, Test Score: 0.062003317471633394, Time Cost: 0.002007007598876953
Epoch: 300, Test Loss: 0.05571487918496132, Test Score: 0.055714879689520234, Time Cost: 0.0016281604766845703
Epoch: 400, Test Loss: 0.05322619900107384, Test Score: 0.05322619494837182, Time Cost: 0.0015711784362792969
Epoch: 500, Test Loss: 0.05162668600678444, Test Score: 0.05162668252867226, Time Cost: 0.0015790462493896484
Epoch: 600, Test Loss: 0.05041322857141495, Test Score: 0.050413228185061956, Time Cost: 0.0016319751739501953
Epoch: 700, Test Loss: 0.04955032095313072, Test Score: 0.049550317632046405, Time Cost: 0.001790761947631836
Epoch: 800, Test Loss: 0.04900002107024193, Test Score: 0.04900001848713028, Time Cost: 0.00161623954772949

In [6]:
print('testing model...')
test_result = learner_obj.test(model=model_obj, test_loader=data_loader['test_loader'], device=device,
                               metric=metric_obj)

testing model...


In [7]:
print('evaluating result...')
print(metric_obj.__class__.__name__, metric_obj.evaluate(y_true=test_result['y_true'], y_pred=test_result['y_pred'], y_score=test_result['y_score'], ))

evaluating result...
mse 0.04859122314268706


In [8]:
from tinybig.metric import accuracy

y_rounded_label = [[round(y[0])] for y in test_result['y_score']]
test_result['y_pred'] = y_rounded_label

acc_metric = accuracy('accuracy_metric')
print('evaluating rounded prediction labels...')
print(acc_metric.__class__.__name__, acc_metric.evaluate(y_true=test_result['y_true'], y_pred=test_result['y_pred'], y_score=test_result['y_score'], ))

evaluating rounded prediction labels...
accuracy 1.0


In [9]:
print('saving result...')
result_obj.save(test_result, cache_dir='./result', output_file='{}_result'.format(config_file_name))
result_obj.save(training_records, cache_dir='./result', output_file='{}_record'.format(config_file_name))

saving result...
